In [1]:
import numpy as np
import extracted_data_reader
from tsai.all import *

In [2]:
# initialization variables
show_splitting_dist = False
show_images = True
find_best_lr = False

ex_number = 1
frame_number = "max" # mean, max or min
target = "goal" # goal, width, head, shoulders or trunk
epochs_number = 20
batch_size = 32

In [ ]:
X = np.array(extracted_data_reader.read_data_ex(ex_number=ex_number, frame_number=frame_number))
Y = np.array(extracted_data_reader.read_target_ex(ex_number=ex_number))
y = np.array([y[:5] for y in Y])

X.shape, y.shape

In [ ]:
splits = get_splits(y, n_splits=len(y), shuffle=False, show_plot=show_splitting_dist)

In [ ]:
tfms = [None, TSClassification()]
datasets = []
for split in splits:
    datasets.append(TSDatasets(X, y, splits=split, tfms=tfms, inplace=True))

In [ ]:
dls = TSDataLoaders.from_dsets(*datasets, bs=batch_size, batch_tfms=[TSStandardize()], num_workers=0)
if show_images: dls.show_batch(sharey=True)

In [ ]:
learn = ts_learner(dls, 'InceptionTimePlus', metrics=accuracy)

In [ ]:
learn.fit_one_cycle(epochs_number, 1e-3)

In [ ]:
image_base_name = f"C:/Users/Gabriele/Downloads/Uni/IOR-Rehab-Thesis/images/newmodels/ex{ex_number}/ex{ex_number}_{frame_number}"
learn.recorder.plot_metrics(figname=f"{image_base_name}_metrics.png")
learn.show_results(figname=f"{image_base_name}_results.png")
# learn.show_probas(figname=f"{image_base_name}_probas.png")
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figname=f"{image_base_name}_confusion_matrix.png")

In [3]:
X_test = np.array(extracted_data_reader.read_data_ex(ex_number, frame_number, test=True))
# print(X_test.shape, X.shape)
# probas, target, preds = learn.get_X_preds(X_test)
# preds

(6, 18, 380) max


In [ ]:
learn.export(f"models_without_tab/ex{ex_number}_{frame_number}.pkl")

In [4]:
learn1 = load_learner(f"models_without_tab/ex{ex_number}_{frame_number}.pkl")
probas, target, preds = learn1.get_X_preds(X_test)
preds

array([[4, 3, 5, 4, 5],
       [4, 3, 5, 4, 5],
       [4, 5, 5, 4, 5],
       [4, 5, 5, 4, 5],
       [5, 4, 5, 4, 4],
       [5, 2, 3, 3, 4]])

In [ ]:
learn=load_learner("models_export/ex2/max_ex2_goal_learner.pkl")